In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

the_blind_flight_synapse_drive_ps_1_path = kagglehub.competition_download('the-blind-flight-synapse-drive-ps-1')
championsproo_subfill4_path = kagglehub.dataset_download('championsproo/subfill4')
championsproo_blindflightdesert_pytorch_default_1_path = kagglehub.model_download('championsproo/blindflightdesert/PyTorch/default/1')
championsproo_blindflightterrain_pytorch_default_1_path = kagglehub.model_download('championsproo/blindflightterrain/PyTorch/default/1')
championsproo_blindflightforest_pytorch_default_1_path = kagglehub.model_download('championsproo/blindflightforest/PyTorch/default/1')
championsproo_blindflightlab_pytorch_default_1_path = kagglehub.model_download('championsproo/blindflightlab/PyTorch/default/1')
championsproo_blindflightskewst_pytorch_default_1_path = kagglehub.model_download('championsproo/blindflightskewst/PyTorch/default/1')

print('Data source import complete.')


In [ ]:
!pip uninstall -y numpy opencv-python opencv-python-headless pandas albumentations segmentation-models-pytorch timm
!pip install \
    numpy==1.26.4 \
    opencv-python-headless \
    pandas \
    albumentations \
    timm \
    segmentation-models-pytorch==0.3.3 \
    --quiet


In [ ]:
import os, random, numpy as np, cv2, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image, ImageDraw
import segmentation_models_pytorch as smp
from pathlib import Path
from tqdm import tqdm

# ---------------- CONFIGURATION ----------------
# CHANGE THIS ID TO TRAIN DIFFERENT SPECIALISTS!
# 1 = Forest Specialist
# 2 = Desert Specialist
TARGET_TERRAIN_ID = 2
MODEL_NAME = "unet_forest_specialist.pth" if TARGET_TERRAIN_ID == 1 else "unet_desert_specialist.pth"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
GRID_SIZE = 20
TILE_SIZE = 40
IMG_SIZE = 800
BATCH_SIZE = 32
ACCUM_STEPS = 4

# ---------------- ASSETS (Same as before) ----------------
INPUT_ROOT = Path("/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset")
ASSET_ROOT = next((p for p in INPUT_ROOT.rglob("assets") if p.is_dir()), Path("assets"))

def load_asset(path):
    try: return Image.open(path).convert("RGBA").resize((TILE_SIZE, TILE_SIZE))
    except: return Image.new("RGBA", (TILE_SIZE, TILE_SIZE), (128,128,128,255))

# We only strictly need the assets for the target ID, but loading all is safer for code compat
ASSETS = {
    0: {"floor":[load_asset(ASSET_ROOT/"lab/t2_floor.png")], "wall":[load_asset(ASSET_ROOT/"lab/t2_wall.png")], "hazard":[load_asset(ASSET_ROOT/"lab/t2_glue.png")], "start":load_asset(ASSET_ROOT/"lab/t2_drone.png"), "goal":load_asset(ASSET_ROOT/"lab/t2_goal.png")},
    1: {"floor":[load_asset(ASSET_ROOT/"forest/t0_dirt.png")], "wall":[load_asset(ASSET_ROOT/"forest/t0_tree.png")], "hazard":[load_asset(ASSET_ROOT/"forest/t0_puddle.png")], "start":load_asset(ASSET_ROOT/"forest/t0_startship.png"), "goal":load_asset(ASSET_ROOT/"forest/t0_goal.png")},
    2: {"floor":[load_asset(ASSET_ROOT/"desert/t1_sand.png")], "wall":[load_asset(ASSET_ROOT/"desert/t1_rocks.png")], "hazard":[load_asset(ASSET_ROOT/"desert/t1_quicksand.png")], "start":load_asset(ASSET_ROOT/"desert/t1_rover.png"), "goal":load_asset(ASSET_ROOT/"desert/t1_goal.png")}
}

# ---------------- SPECIALIST DATASET ----------------
class SpecialistDataset(Dataset):
    def __init__(self, length=10000, target_id=1):
        self.length = length
        self.target_id = target_id # FORCES this terrain only
        self.color = A.Compose([
            A.HueSaturationValue(5, 10, 5, p=0.4),
            A.RGBShift(10, 10, 10, p=0.3),
            A.RandomBrightnessContrast(0.1, 0.1, p=0.4),
        ])
        self.blur = A.OneOf([A.GaussianBlur((3, 5), p=1.0), A.MotionBlur((3, 7), p=1.0)], p=0.4)
        self.final = A.Compose([A.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), ToTensorV2()])

    def __len__(self): return self.length

    def mild_skew(self, img, mask):
        if random.random() > 0.4: return img, mask
        h, w = img.shape[:2]
        limit = 40
        pts1 = np.float32([[0, 0], [w, 0], [0, h], [w, h]])
        pts2 = np.float32([
            [random.randint(0, limit), random.randint(0, limit)],
            [w - random.randint(0, limit), random.randint(0, limit)],
            [random.randint(0, limit), h - random.randint(0, limit)],
            [w - random.randint(0, limit), h - random.randint(0, limit)]
        ])
        M = cv2.getPerspectiveTransform(pts1, pts2)
        img = cv2.warpPerspective(img, M, (w, h), borderMode=cv2.BORDER_REFLECT_101)
        mask = cv2.warpPerspective(mask, M, (w, h), flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_REFLECT_101)
        return img, mask

    def __getitem__(self, idx):
        # --- SPECIALIST LOGIC: ONLY GENERATE TARGET TERRAIN ---
        t_id = self.target_id
        assets = ASSETS[t_id]

        grid = np.random.choice([0, 1, 2], (GRID_SIZE, GRID_SIZE), p=[0.7, 0.2, 0.1])
        s, g = random.sample([(r, c) for r in range(GRID_SIZE) for c in range(GRID_SIZE)], 2)
        grid[s] = 3; grid[g] = 4
        k = random.choice([0, 1, 2, 3])
        if k > 0: grid = np.rot90(grid, k=k)

        canvas = Image.new("RGBA", (IMG_SIZE, IMG_SIZE), (0, 0, 0, 255))
        draw = ImageDraw.Draw(canvas)

        for r in range(GRID_SIZE):
            for c in range(GRID_SIZE):
                x, y = c * TILE_SIZE, r * TILE_SIZE
                canvas.paste(random.choice(assets["floor"]), (x, y))
                cell = grid[r, c]
                if cell == 1: canvas.paste(random.choice(assets["wall"]), (x, y))
                elif cell == 2: canvas.paste(random.choice(assets["hazard"]), (x, y))
                elif cell == 3: canvas.paste(assets["start"], (x, y))
                elif cell == 4: canvas.paste(assets["goal"], (x, y))
                draw.rectangle([x, y, x + TILE_SIZE - 1, y + TILE_SIZE - 1], outline=(0, 0, 0), width=2)

        img = np.array(canvas.convert("RGB"))
        mask_full = cv2.resize(grid.astype(np.uint8), (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
        img, mask_full = self.mild_skew(img, mask_full)

        img = self.color(image=img)["image"]
        img = self.blur(image=img)["image"]
        img = self.final(image=img)["image"]

        mask = cv2.resize(mask_full, (GRID_SIZE, GRID_SIZE), interpolation=cv2.INTER_NEAREST)

        # We don't need to return t_id because we know what it is,
        # but keeping format consistent with your pipelines
        return img, torch.from_numpy(mask).long()

# ---------------- MODEL (Standard UNet) ----------------
# Specialists don't need the terrain head, just the segmentation head
class SpecialistUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.unet = smp.Unet(encoder_name="resnet34", encoder_weights="imagenet", in_channels=3, classes=5)

    def forward(self, x):
        # Direct segmentation, no terrain auxiliary output
        return self.unet(x)

# ---------------- TRAIN ----------------
def train_specialist():
    # Load Specialist Dataset
    ds = SpecialistDataset(length=10000, target_id=TARGET_TERRAIN_ID)
    dl = DataLoader(ds, BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

    model = SpecialistUNet().to(DEVICE)
    if torch.cuda.device_count() > 1: model = nn.DataParallel(model)

    opt = optim.AdamW(model.parameters(), 1e-3, weight_decay=1e-2)
    scaler = GradScaler()
    sched = optim.lr_scheduler.ReduceLROnPlateau(opt, "min", 0.5, 2)

    # Loss for segmentation only
    ce = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 5.0, 5.0, 50.0, 50.0], device=DEVICE))

    print(f"🚀 Starting Specialist Training for ID {TARGET_TERRAIN_ID}...")
    print(f"💾 Saving to: {MODEL_NAME}")

    for ep in range(12): # 12 Epochs is usually enough for specialized tasks
        model.train()
        loop = tqdm(dl, desc=f"Epoch {ep+1}/12")
        epoch_loss = 0

        for i, (x, y) in enumerate(loop):
            x, y = x.to(DEVICE), y.to(DEVICE)

            with autocast():
                # Note: Model output is raw logits here because we removed interpolation in model class
                # We interpolate logits to match label size
                logits = model(x)
                logits = torch.nn.functional.interpolate(logits, (GRID_SIZE, GRID_SIZE), mode='bilinear', align_corners=False)

                loss = ce(logits, y) / ACCUM_STEPS

            scaler.scale(loss).backward()

            if (i+1) % ACCUM_STEPS == 0:
                scaler.step(opt); scaler.update(); opt.zero_grad()
                epoch_loss += loss.item() * ACCUM_STEPS
                loop.set_postfix({"Loss": f"{loss.item()*ACCUM_STEPS:.4f}"})

        sched.step(epoch_loss/len(dl))

    torch.save(model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict(), MODEL_NAME)
    print(f"✅ {MODEL_NAME} Saved Successfully!")

if __name__=="__main__":
    train_specialist()

In [ ]:
# ---------------- TERRAIN CLASSIFIER CONFIG ----------------
CLASSIFIER_MODEL_NAME = "terrain_classifier.pth"

# ---------------- CLASSIFIER DATASET (Randomized) ----------------
# This dataset MUST generate all 3 terrains to teach the classifier the difference
class ClassifierDataset(Dataset):
    def __init__(self, length=10000):
        self.length = length
        self.specialist_ds = SpecialistDataset(length=1) # Helper to reuse logic

    def __len__(self): return self.length

    def __getitem__(self, idx):
        # Randomly pick a terrain
        t_id = random.choice([0, 1, 2])

        # Temporarily switch the helper's target_id
        self.specialist_ds.target_id = t_id

        # Get image (we ignore the mask, we only need the label t_id)
        img, _ = self.specialist_ds.__getitem__(0)

        return img, torch.tensor(t_id).long()

# ---------------- CLASSIFIER MODEL ----------------
class TerrainRouter(nn.Module):
    def __init__(self):
        super().__init__()
        # Load a pretrained ResNet, remove the FC layer, add our own
        self.backbone = smp.encoders.get_encoder("resnet34", in_channels=3, weights="imagenet")
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 3) # 3 Classes: Lab, Forest, Desert
        )

    def forward(self, x):
        feats = self.backbone(x)
        # ResNet34 features are a list, last one is the deepest (512 channels)
        x = feats[-1]
        x = self.pool(x)
        return self.head(x)

# ---------------- TRAIN ROUTER ----------------
def train_router():
    dl = DataLoader(ClassifierDataset(), BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    model = TerrainRouter().to(DEVICE)

    opt = optim.Adam(model.parameters(), lr=1e-4)
    loss_fn = nn.CrossEntropyLoss()

    print(f"🚀 Starting Terrain Router Training...")

    for ep in range(5): # Classifiers learn very fast, 5 epochs is plenty
        model.train()
        loop = tqdm(dl, desc=f"Router Epoch {ep+1}/5")

        for x, t in loop:
            x, t = x.to(DEVICE), t.to(DEVICE)

            opt.zero_grad()
            logits = model(x)
            loss = loss_fn(logits, t)
            loss.backward()
            opt.step()

            loop.set_postfix({"Acc": f"{(logits.argmax(1) == t).float().mean():.2f}", "Loss": f"{loss.item():.4f}"})

    torch.save(model.state_dict(), CLASSIFIER_MODEL_NAME)
    print(f"✅ {CLASSIFIER_MODEL_NAME} Saved!")

if __name__=="__main__":
    train_router()

In [ ]:
import os, random, numpy as np, cv2, torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image, ImageDraw
import segmentation_models_pytorch as smp
from pathlib import Path
from tqdm import tqdm

# ---------------- CONFIGURATION ----------------
# CHANGE THIS ID TO TRAIN DIFFERENT SPECIALISTS!
# 1 = Forest Specialist
# 2 = Desert Specialist
TARGET_TERRAIN_ID = 1
MODEL_NAME = "unet_forest_specialist.pth" if TARGET_TERRAIN_ID == 1 else "unet_desert_specialist.pth"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
GRID_SIZE = 20
TILE_SIZE = 40
IMG_SIZE = 800
BATCH_SIZE = 32
ACCUM_STEPS = 4

# ---------------- ASSETS (Same as before) ----------------
INPUT_ROOT = Path("/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset")
ASSET_ROOT = next((p for p in INPUT_ROOT.rglob("assets") if p.is_dir()), Path("assets"))

def load_asset(path):
    try: return Image.open(path).convert("RGBA").resize((TILE_SIZE, TILE_SIZE))
    except: return Image.new("RGBA", (TILE_SIZE, TILE_SIZE), (128,128,128,255))

# We only strictly need the assets for the target ID, but loading all is safer for code compat
ASSETS = {
    0: {"floor":[load_asset(ASSET_ROOT/"lab/t2_floor.png")], "wall":[load_asset(ASSET_ROOT/"lab/t2_wall.png")], "hazard":[load_asset(ASSET_ROOT/"lab/t2_glue.png")], "start":load_asset(ASSET_ROOT/"lab/t2_drone.png"), "goal":load_asset(ASSET_ROOT/"lab/t2_goal.png")},
    1: {"floor":[load_asset(ASSET_ROOT/"forest/t0_dirt.png")], "wall":[load_asset(ASSET_ROOT/"forest/t0_tree.png")], "hazard":[load_asset(ASSET_ROOT/"forest/t0_puddle.png")], "start":load_asset(ASSET_ROOT/"forest/t0_startship.png"), "goal":load_asset(ASSET_ROOT/"forest/t0_goal.png")},
    2: {"floor":[load_asset(ASSET_ROOT/"desert/t1_sand.png")], "wall":[load_asset(ASSET_ROOT/"desert/t1_rocks.png")], "hazard":[load_asset(ASSET_ROOT/"desert/t1_quicksand.png")], "start":load_asset(ASSET_ROOT/"desert/t1_rover.png"), "goal":load_asset(ASSET_ROOT/"desert/t1_goal.png")}
}

# ---------------- SPECIALIST DATASET ----------------
class SpecialistDataset(Dataset):
    def __init__(self, length=10000, target_id=1):
        self.length = length
        self.target_id = target_id # FORCES this terrain only
        self.color = A.Compose([
            A.HueSaturationValue(5, 10, 5, p=0.4),
            A.RGBShift(10, 10, 10, p=0.3),
            A.RandomBrightnessContrast(0.1, 0.1, p=0.4),
        ])
        self.blur = A.OneOf([A.GaussianBlur((3, 5), p=1.0), A.MotionBlur((3, 7), p=1.0)], p=0.4)
        self.final = A.Compose([A.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), ToTensorV2()])

    def __len__(self): return self.length

    def mild_skew(self, img, mask):
        if random.random() > 0.4: return img, mask
        h, w = img.shape[:2]
        limit = 40
        pts1 = np.float32([[0, 0], [w, 0], [0, h], [w, h]])
        pts2 = np.float32([
            [random.randint(0, limit), random.randint(0, limit)],
            [w - random.randint(0, limit), random.randint(0, limit)],
            [random.randint(0, limit), h - random.randint(0, limit)],
            [w - random.randint(0, limit), h - random.randint(0, limit)]
        ])
        M = cv2.getPerspectiveTransform(pts1, pts2)
        img = cv2.warpPerspective(img, M, (w, h), borderMode=cv2.BORDER_REFLECT_101)
        mask = cv2.warpPerspective(mask, M, (w, h), flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_REFLECT_101)
        return img, mask

    def __getitem__(self, idx):
        # --- SPECIALIST LOGIC: ONLY GENERATE TARGET TERRAIN ---
        t_id = self.target_id
        assets = ASSETS[t_id]

        grid = np.random.choice([0, 1, 2], (GRID_SIZE, GRID_SIZE), p=[0.7, 0.2, 0.1])
        s, g = random.sample([(r, c) for r in range(GRID_SIZE) for c in range(GRID_SIZE)], 2)
        grid[s] = 3; grid[g] = 4
        k = random.choice([0, 1, 2, 3])
        if k > 0: grid = np.rot90(grid, k=k)

        canvas = Image.new("RGBA", (IMG_SIZE, IMG_SIZE), (0, 0, 0, 255))
        draw = ImageDraw.Draw(canvas)

        for r in range(GRID_SIZE):
            for c in range(GRID_SIZE):
                x, y = c * TILE_SIZE, r * TILE_SIZE
                canvas.paste(random.choice(assets["floor"]), (x, y))
                cell = grid[r, c]
                if cell == 1: canvas.paste(random.choice(assets["wall"]), (x, y))
                elif cell == 2: canvas.paste(random.choice(assets["hazard"]), (x, y))
                elif cell == 3: canvas.paste(assets["start"], (x, y))
                elif cell == 4: canvas.paste(assets["goal"], (x, y))
                draw.rectangle([x, y, x + TILE_SIZE - 1, y + TILE_SIZE - 1], outline=(0, 0, 0), width=2)

        img = np.array(canvas.convert("RGB"))
        mask_full = cv2.resize(grid.astype(np.uint8), (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
        img, mask_full = self.mild_skew(img, mask_full)

        img = self.color(image=img)["image"]
        img = self.blur(image=img)["image"]
        img = self.final(image=img)["image"]

        mask = cv2.resize(mask_full, (GRID_SIZE, GRID_SIZE), interpolation=cv2.INTER_NEAREST)

        # We don't need to return t_id because we know what it is,
        # but keeping format consistent with your pipelines
        return img, torch.from_numpy(mask).long()

# ---------------- MODEL (Standard UNet) ----------------
# Specialists don't need the terrain head, just the segmentation head
class SpecialistUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.unet = smp.Unet(encoder_name="resnet34", encoder_weights="imagenet", in_channels=3, classes=5)

    def forward(self, x):
        # Direct segmentation, no terrain auxiliary output
        return self.unet(x)

# ---------------- TRAIN ----------------
def train_specialist():
    # Load Specialist Dataset
    ds = SpecialistDataset(length=10000, target_id=TARGET_TERRAIN_ID)
    dl = DataLoader(ds, BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

    model = SpecialistUNet().to(DEVICE)
    if torch.cuda.device_count() > 1: model = nn.DataParallel(model)

    opt = optim.AdamW(model.parameters(), 1e-3, weight_decay=1e-2)
    scaler = GradScaler()
    sched = optim.lr_scheduler.ReduceLROnPlateau(opt, "min", 0.5, 2)

    # Loss for segmentation only
    ce = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 5.0, 5.0, 50.0, 50.0], device=DEVICE))

    print(f"🚀 Starting Specialist Training for ID {TARGET_TERRAIN_ID}...")
    print(f"💾 Saving to: {MODEL_NAME}")

    for ep in range(12): # 12 Epochs is usually enough for specialized tasks
        model.train()
        loop = tqdm(dl, desc=f"Epoch {ep+1}/12")
        epoch_loss = 0

        for i, (x, y) in enumerate(loop):
            x, y = x.to(DEVICE), y.to(DEVICE)

            with autocast():
                # Note: Model output is raw logits here because we removed interpolation in model class
                # We interpolate logits to match label size
                logits = model(x)
                logits = torch.nn.functional.interpolate(logits, (GRID_SIZE, GRID_SIZE), mode='bilinear', align_corners=False)

                loss = ce(logits, y) / ACCUM_STEPS

            scaler.scale(loss).backward()

            if (i+1) % ACCUM_STEPS == 0:
                scaler.step(opt); scaler.update(); opt.zero_grad()
                epoch_loss += loss.item() * ACCUM_STEPS
                loop.set_postfix({"Loss": f"{loss.item()*ACCUM_STEPS:.4f}"})

        sched.step(epoch_loss/len(dl))

    torch.save(model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict(), MODEL_NAME)
    print(f"✅ {MODEL_NAME} Saved Successfully!")

if __name__=="__main__":
    train_specialist()

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
import json
import heapq
import random
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pathlib import Path
from PIL import Image

# --- CONFIGURATION ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# FILES
LAB_PATH     = "/kaggle/input/blindflightlab/pytorch/default/1/lab_specialist_final.pth"
FOREST_PATH  = "/kaggle/input/blindflightforest/pytorch/default/1/unet_forest_specialist.pth"
DESERT_PATH  = "/kaggle/input/blindflightdesert/pytorch/default/1/unet_desert_specialist.pth"
ROUTER_PATH  = "/kaggle/input/blindflightterrain/pytorch/default/1/terrain_classifier.pth"

TEST_IMG_DIR = Path("/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/test/images")
TEST_VEL_DIR = Path("/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/test/velocities")

# --- MODEL CLASSES ---
class MultiHeadUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.unet = smp.Unet(encoder_name="resnet34", encoder_weights=None, in_channels=3, classes=5)
        c = self.unet.encoder.out_channels[-1]
        self.terrain = nn.Sequential(nn.AdaptiveAvgPool2d(1), nn.Flatten(), nn.Linear(c, 64), nn.ReLU(), nn.Linear(64, 3))
    def forward(self, x): return self.unet(x), self.terrain(self.unet.encoder(x)[-1])

class SpecialistUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.unet = smp.Unet(encoder_name="resnet34", encoder_weights=None, in_channels=3, classes=5)
    def forward(self, x): return self.unet(x)

class TerrainRouter(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = smp.encoders.get_encoder("resnet34", in_channels=3, weights=None)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.head = nn.Sequential(nn.Flatten(), nn.Linear(512, 128), nn.ReLU(), nn.Dropout(0.2), nn.Linear(128, 3))
    def forward(self, x): return self.head(self.pool(self.backbone(x)[-1]))

def load_weights_smart(model, path):
    if not Path(path).exists(): return False
    state = torch.load(path, map_location="cpu")
    if list(state.keys())[0].startswith('module.'): state = {k[7:]: v for k, v in state.items()}
    model_keys = list(model.state_dict().keys())
    file_keys = list(state.keys())
    if model_keys[0].startswith('unet.') and not file_keys[0].startswith('unet.'):
        state = {f"unet.{k}": v for k, v in state.items()}
    elif not model_keys[0].startswith('unet.') and file_keys[0].startswith('unet.'):
        state = {k[5:]: v for k, v in state.items() if k.startswith('unet.')}
    try: model.load_state_dict(state, strict=False); return True
    except: return False

# --- PATHFINDER (Probability Based) ---
def solve_path(grid, probs, boost, t_id):
    costs = {
        0: {0: 1.0, 1: float('inf'), 2: 3.0, 3: 1.0, 4: 2.0},
        1: {0: 1.5, 1: float('inf'), 2: 2.8, 3: 1.5, 4: 2.5},
        2: {0: 1.2, 1: float('inf'), 2: 3.7, 3: 1.2, 4: 2.2}
    }[t_id]

    starts, goals = np.argwhere(grid == 3), np.argwhere(grid == 4)
    if len(starts) == 0 or len(goals) == 0: return "NO PATH", []

    start = tuple(max(starts, key=lambda p: probs[3, p[0], p[1]]))
    goal = tuple(max(goals, key=lambda p: probs[4, p[0], p[1]]))

    pq = [(0, 0, start)]
    g_score = {start: 0.0}
    came_from = {start: None}

    while pq:
        _, curr_g, curr = heapq.heappop(pq)
        if curr_g > g_score.get(curr, float('inf')): continue
        if curr == goal: break

        r, c = curr
        for next_node, move in [((r-1,c),'u'),((r+1,c),'d'),((r,c-1),'l'),((r,c+1),'r')]:
            nr, nc = next_node
            if 0 <= nr < 20 and 0 <= nc < 20 and grid[nr, nc] != 1:
                step_cost = max(costs.get(grid[nr,nc], 1.0) - boost[nr,nc], 0.01)
                new_g = curr_g + step_cost
                if new_g < g_score.get(next_node, float('inf')):
                    g_score[next_node] = new_g
                    came_from[next_node] = (curr, move)
                    h = abs(nr - goal[0]) + abs(nc - goal[1])
                    heapq.heappush(pq, (new_g + h, new_g, next_node))

    if goal not in came_from: return "NO PATH", []
    path_str, coords, curr = [], [], goal
    while curr != start:
        prev, move = came_from[curr]
        path_str.append(move)
        coords.append(curr)
        curr = prev
    coords.append(start)
    return "".join(path_str[::-1]), coords[::-1]

# --- VISUALIZER ---
def visualize_ensemble(num_samples=5):
    models = {
        'router': TerrainRouter().to(DEVICE),
        0: MultiHeadUNet().to(DEVICE), 1: SpecialistUNet().to(DEVICE), 2: SpecialistUNet().to(DEVICE)
    }
    print("⏳ Loading Models...")
    load_weights_smart(models['router'], ROUTER_PATH)
    load_weights_smart(models[0], LAB_PATH)
    load_weights_smart(models[1], FOREST_PATH)
    load_weights_smart(models[2], DESERT_PATH)
    for m in models.values(): m.eval()

    transform = A.Compose([A.Resize(800, 800), A.Normalize(), ToTensorV2()])
    files = list(TEST_IMG_DIR.glob("*.png"))
    samples = random.sample(files, min(len(files), num_samples))

    for img_path in samples:
        raw_img = np.array(Image.open(img_path).convert("RGB"))
        x = transform(image=raw_img)['image'].unsqueeze(0).to(DEVICE)

        with torch.no_grad():
            router_out = models['router'](x)
            t_id = torch.argmax(router_out, dim=1).item()
            t_name = ["LAB", "FOREST", "DESERT"][t_id]
            spec_out = models[t_id](x)
            seg_logits = spec_out[0] if isinstance(spec_out, tuple) else spec_out
            if seg_logits.shape[-1] != 20:
                seg_logits = torch.nn.functional.interpolate(seg_logits, (20, 20), mode='bilinear', align_corners=False)

            probs = torch.softmax(seg_logits, dim=1).squeeze().cpu().numpy()
            grid = np.argmax(probs, axis=0)

        boost = np.zeros((20, 20))
        path_str, coords = solve_path(grid, probs, boost, t_id)

        # --- PLOT ---
        fig, ax = plt.subplots(1, 2, figsize=(16, 8))

        # LEFT: Real Image
        ax[0].imshow(raw_img)
        ax[0].set_title(f"Terrain: {t_name}", fontsize=14)

        # Calculate Coordinates (Precise Center of 40x40 tiles)
        sy, sx = raw_img.shape[0]/20, raw_img.shape[1]/20

        # Overlay a faint Grid to prove alignment logic
        # Vertical lines
        for i in range(1, 20):
            ax[0].axvline(x=i*sx, color='white', linewidth=0.5, alpha=0.3)
            ax[0].axhline(y=i*sy, color='white', linewidth=0.5, alpha=0.3)

        if coords:
            py, px = zip(*coords)
            plot_px = [(c+0.5)*sx for c in px]
            plot_py = [(r+0.5)*sy for r in py]

            # Simple, Thin, Red Line
            ax[0].plot(plot_px, plot_py, color='red', linewidth=2, label='Path')

            # Standard Markers
            ax[0].scatter(plot_px[0], plot_py[0], c='lime', s=80, edgecolors='black', label='Start', zorder=5)
            ax[0].scatter(plot_px[-1], plot_py[-1], c='magenta', s=80, edgecolors='black', label='Goal', zorder=5)
            ax[0].legend(loc='lower left')
        else:
            ax[0].text(400, 400, "NO PATH FOUND", c='red', fontsize=16, ha='center', backgroundcolor='black')

        ax[0].axis('off')

        # RIGHT: Logic View
        cmap = plt.get_cmap('tab10', 5)
        im = ax[1].imshow(grid, cmap=cmap, vmin=-0.5, vmax=4.5)
        ax[1].set_title("Logic View (20x20)", fontsize=14)

        cbar = plt.colorbar(im, ax=ax[1], ticks=[0,1,2,3,4], fraction=0.046, pad=0.04)
        cbar.ax.set_yticklabels(['Floor', 'Wall', 'Haz', 'Start', 'Goal'], fontsize=10)

        plt.tight_layout()
        plt.show()

if __name__ == "__main__":
    visualize_ensemble(15)

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import json
import heapq
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset, DataLoader

# --- CONFIGURATION ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_MULTI_GPU = torch.cuda.device_count() > 1

# MODEL PATHS
LAB_PATH     = "/kaggle/input/blindflightlab/pytorch/default/1/lab_specialist_final.pth"
FOREST_PATH  = "/kaggle/input/blindflightforest/pytorch/default/1/unet_forest_specialist.pth"
DESERT_PATH  = "/kaggle/input/blindflightdesert/pytorch/default/1/unet_desert_specialist.pth"
ROUTER_PATH  = "/kaggle/input/blindflightterrain/pytorch/default/1/terrain_classifier.pth"

# DATA PATHS
TEST_IMG_DIR = Path("/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/test/images")
TEST_VEL_DIR = Path("/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset/test/velocities")
OUTPUT_CSV = "submission.csv"

# BATCH PROCESSING
BATCH_SIZE = 8  # Process multiple images simultaneously

# --- MODEL CLASSES ---
class MultiHeadUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.unet = smp.Unet(encoder_name="resnet34", encoder_weights=None, in_channels=3, classes=5)
        c = self.unet.encoder.out_channels[-1]
        self.terrain = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), nn.Flatten(),
            nn.Linear(c, 64), nn.ReLU(), nn.Linear(64, 3)
        )
    def forward(self, x):
        return self.unet(x), self.terrain(self.unet.encoder(x)[-1])

class SpecialistUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.unet = smp.Unet(encoder_name="resnet34", encoder_weights=None, in_channels=3, classes=5)
    def forward(self, x):
        return self.unet(x)

class TerrainRouter(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = smp.encoders.get_encoder("resnet34", in_channels=3, weights=None)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.head = nn.Sequential(
            nn.Flatten(), nn.Linear(512, 128), nn.ReLU(),
            nn.Dropout(0.2), nn.Linear(128, 3)
        )
    def forward(self, x):
        return self.head(self.pool(self.backbone(x)[-1]))

def load_weights_smart(model, path):
    """Smart weight loader handling module prefixes"""
    if not Path(path).exists():
        print(f"⚠️ Warning: {path} not found")
        return False

    state = torch.load(path, map_location="cpu")

    # Remove 'module.' prefix if present (DataParallel)
    if list(state.keys())[0].startswith('module.'):
        state = {k[7:]: v for k, v in state.items()}

    # Handle 'unet.' prefix mismatches
    model_keys = list(model.state_dict().keys())
    file_keys = list(state.keys())

    if model_keys[0].startswith('unet.') and not file_keys[0].startswith('unet.'):
        state = {f"unet.{k}": v for k, v in state.items()}
    elif not model_keys[0].startswith('unet.') and file_keys[0].startswith('unet.'):
        state = {k[5:]: v for k, v in state.items() if k.startswith('unet.')}

    try:
        model.load_state_dict(state, strict=False)
        return True
    except Exception as e:
        print(f"⚠️ Error loading {path}: {e}")
        return False

# --- A* PATHFINDER ---
def solve_path(grid, probs, boost, t_id):
    """
    A* pathfinding with velocity boost consideration
    Returns: (path_string, coordinates_list)
    """
    # Terrain-specific base costs
    costs = {
        0: {0: 1.0, 1: float('inf'), 2: 3.0, 3: 1.0, 4: 2.0},  # Lab
        1: {0: 1.5, 1: float('inf'), 2: 2.8, 3: 1.5, 4: 2.5},  # Forest
        2: {0: 1.2, 1: float('inf'), 2: 3.7, 3: 1.2, 4: 2.2}   # Desert
    }[t_id]

    # Find START and GOAL using probability-weighted selection
    starts = np.argwhere(grid == 3)
    goals = np.argwhere(grid == 4)

    if len(starts) == 0 or len(goals) == 0:
        return "", []

    # Pick highest confidence START/GOAL
    start = tuple(max(starts, key=lambda p: probs[3, p[0], p[1]]))
    goal = tuple(max(goals, key=lambda p: probs[4, p[0], p[1]]))

    # A* Search
    pq = [(0, 0, start)]  # (f_score, g_score, node)
    g_score = {start: 0.0}
    came_from = {start: None}

    while pq:
        _, curr_g, curr = heapq.heappop(pq)

        if curr_g > g_score.get(curr, float('inf')):
            continue

        if curr == goal:
            break

        r, c = curr
        # Explore 4-connected neighbors
        for next_node, move in [((r-1,c),'u'), ((r+1,c),'d'),
                                 ((r,c-1),'l'), ((r,c+1),'r')]:
            nr, nc = next_node

            if 0 <= nr < 20 and 0 <= nc < 20 and grid[nr, nc] != 1:
                # Calculate step cost with boost
                base = costs.get(grid[nr, nc], 1.0)
                step_cost = max(base - boost[nr, nc], 0.01)

                new_g = curr_g + step_cost

                if new_g < g_score.get(next_node, float('inf')):
                    g_score[next_node] = new_g
                    came_from[next_node] = (curr, move)

                    # Manhattan heuristic
                    h = abs(nr - goal[0]) + abs(nc - goal[1])
                    heapq.heappush(pq, (new_g + h, new_g, next_node))

    # Reconstruct path
    if goal not in came_from:
        return "", []

    path_str = []
    coords = []
    curr = goal

    while curr != start:
        prev, move = came_from[curr]
        path_str.append(move)
        coords.append(curr)
        curr = prev

    coords.append(start)

    return "".join(path_str[::-1]), coords[::-1]

# --- BATCH DATASET ---
class TestDataset(Dataset):
    """Dataset for batch processing test images"""
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image_id = img_path.stem
        raw_img = np.array(Image.open(img_path).convert("RGB"))
        x = self.transform(image=raw_img)['image']

        # Load velocity boost
        vel_path = TEST_VEL_DIR / f"{image_id}.json"
        boost = np.zeros((20, 20))

        if vel_path.exists():
            with open(vel_path, 'r') as f:
                vel_data = json.load(f)
                boost = np.array(vel_data['boost'])

        return x, boost, image_id

# --- MAIN SUBMISSION GENERATOR ---
def generate_submission():
    """Generate submission.csv for all test images"""

    print("🚀 Blind Flight Submission Generator")
    print("=" * 50)

    # Load models
    print("⏳ Loading Models...")
    models = {
        'router': TerrainRouter().to(DEVICE),
        0: MultiHeadUNet().to(DEVICE),
        1: SpecialistUNet().to(DEVICE),
        2: SpecialistUNet().to(DEVICE)
    }

    load_weights_smart(models['router'], ROUTER_PATH)
    load_weights_smart(models[0], LAB_PATH)
    load_weights_smart(models[1], FOREST_PATH)
    load_weights_smart(models[2], DESERT_PATH)

    # Enable DataParallel for multi-GPU
    if USE_MULTI_GPU:
        print(f"🔥 Using {torch.cuda.device_count()} GPUs with DataParallel")
        models['router'] = nn.DataParallel(models['router'])
        models[0] = nn.DataParallel(models[0])
        models[1] = nn.DataParallel(models[1])
        models[2] = nn.DataParallel(models[2])
    else:
        print(f"💻 Using single GPU: {torch.cuda.get_device_name(0)}")

    for m in models.values():
        m.eval()

    print("✅ Models Loaded Successfully")

    # Preprocessing
    transform = A.Compose([
        A.Resize(800, 800),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

    # Get all test images
    img_files = sorted(TEST_IMG_DIR.glob("*.png"))
    print(f"📂 Found {len(img_files)} test images")

    # Create dataset and dataloader
    test_dataset = TestDataset(img_files, transform)
    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    results = []

    # Batch processing with dual GPU
    for batch_imgs, batch_boosts, batch_ids in tqdm(test_loader, desc="Processing Batches"):
        batch_imgs = batch_imgs.to(DEVICE)

        # Inference on batch
        with torch.no_grad():
            # 1. Route to terrain (batch)
            router_out = models['router'](batch_imgs)
            t_ids = torch.argmax(router_out, dim=1).cpu().numpy()

            # 2. Process each terrain type in batch
            for i in range(len(batch_imgs)):
                t_id = int(t_ids[i])
                image_id = batch_ids[i]
                boost = batch_boosts[i].cpu().numpy()

                # Single image inference through specialist
                x_single = batch_imgs[i:i+1]
                spec_out = models[t_id](x_single)
                seg_logits = spec_out[0] if isinstance(spec_out, tuple) else spec_out

                # Ensure 20x20 output
                if seg_logits.shape[-1] != 20:
                    seg_logits = torch.nn.functional.interpolate(
                        seg_logits, (20, 20), mode='bilinear', align_corners=False
                    )

                probs = torch.softmax(seg_logits, dim=1).squeeze().cpu().numpy()
                grid = np.argmax(probs, axis=0)

                # 3. Pathfinding (CPU-bound, runs in parallel with GPU)
                path_str, _ = solve_path(grid, probs, boost, t_id)

                results.append({
                    'image_id': image_id,
                    'path': path_str if path_str else ""
                })

    # Create DataFrame and save
    df = pd.DataFrame(results)
    df.to_csv(OUTPUT_CSV, index=False)

    print(f"\n✅ Submission saved to: {OUTPUT_CSV}")
    print(f"📊 Total samples: {len(df)}")
    print(f"📊 Paths found: {(df['path'] != '').sum()}")
    print(f"📊 No path found: {(df['path'] == '').sum()}")

    # Show sample
    print("\n📋 Sample submissions:")
    print(df.head(10))

    return df

if __name__ == "__main__":
    submission_df = generate_submission()

In [ ]:
import pandas as pd
from pathlib import Path

# --- CONFIGURATION ---
MAIN_SUBMISSION_FILE = "/kaggle/working/submission.csv"        # The file you just generated
FALLBACK_SUBMISSION_FILE = "/kaggle/input/subfill4/submission_filled(4).csv" # <--- UPDATE THIS PATH
OUTPUT_FILENAME = "submis.csv"     # The final file name

def patch_submission():
    # 1. Load Dataframes
    print(f"📂 Loading main file: {MAIN_SUBMISSION_FILE}")
    if not Path(MAIN_SUBMISSION_FILE).exists():
        print("❌ Main submission file not found!")
        return

    df_main = pd.read_csv(MAIN_SUBMISSION_FILE)

    print(f"📂 Loading fallback file: {FALLBACK_SUBMISSION_FILE}")
    if not Path(FALLBACK_SUBMISSION_FILE).exists():
        print("❌ Fallback file not found!")
        return

    df_fallback = pd.read_csv(FALLBACK_SUBMISSION_FILE)

    # 2. Check Column Names (Standardize to 'image_id' and 'path')
    # If your files use 'filename', rename it to 'image_id' for consistency
    if 'filename' in df_main.columns:
        df_main.rename(columns={'filename': 'image_id'}, inplace=True)
    if 'filename' in df_fallback.columns:
        df_fallback.rename(columns={'filename': 'image_id'}, inplace=True)

    # 3. Create a dictionary map from the fallback file
    # { 'image_001.png': 'rrdd...' }
    fallback_map = df_fallback.set_index('image_id')['path'].to_dict()

    # 4. Identify "NO PATH" entries
    # Check for "NO PATH", empty strings, or NaNs
    missing_mask = (df_main['path'] == "NO PATH") | (df_main['path'].isna()) | (df_main['path'] == "")
    missing_count = missing_mask.sum()

    print(f"⚠️ Found {missing_count} rows with 'NO PATH'. Attempting to patch...")

    # 5. Patch Logic
    patched_count = 0

    def fill_path(row):
        nonlocal patched_count
        current_path = row['path']
        img_id = row['image_id']

        # If current path is invalid
        if current_path == "d" or pd.isna(current_path) or current_path == "":
            # Try to find in fallback
            if img_id in fallback_map:
                backup_path = fallback_map[img_id]
                # Only use backup if it is NOT "NO PATH" itself
                if backup_path != "NO PATH" and not pd.isna(backup_path):
                    patched_count += 1
                    return backup_path

        return current_path

    # Apply the patch
    df_main['path'] = df_main.apply(fill_path, axis=1)

    # 6. Save
    df_main.to_csv(OUTPUT_FILENAME, index=False)

    print(f"✅ Patch Complete!")
    print(f"   - Replaced: {patched_count}/{missing_count} missing paths.")
    print(f"   - Remaining 'NO PATH': {(df_main['path'] == 'NO PATH').sum()}")
    print(f"   - Saved to: {OUTPUT_FILENAME}")

if __name__ == "__main__":
    patch_submission()